# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234306444037                   -0.50    8.0         
  2   -7.250285428365       -1.80       -1.40    1.0   5.22ms
  3   -7.250981314898       -3.16       -1.94    1.0   5.26ms
  4   -7.250909365897   +   -4.14       -1.85    2.0   6.51ms
  5   -7.251335126878       -3.37       -2.75    1.0   5.36ms
  6   -7.251337889569       -5.56       -3.12    2.0   6.29ms
  7   -7.251338723075       -6.08       -3.53    2.0   6.60ms
  8   -7.251338783710       -7.22       -3.96    1.0   5.56ms
  9   -7.251338797170       -7.87       -4.52    1.0   5.65ms
 10   -7.251338798474       -8.88       -4.85    2.0   6.78ms
 11   -7.251338798683       -9.68       -5.50    1.0   5.59ms
 12   -7.251338798692      -11.02       -5.45    3.0   7.97ms
 13   -7.251338798703      -10.95       -6.12    1.0   5.56ms
 14   -7.251338798704      -11.94       -6.43    3.0   7.86ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06437802744917334
[ Info: Arnoldi iteration step 2: normres = 0.502556300046487
[ Info: Arnoldi iteration step 3: normres = 0.9247267521698954
[ Info: Arnoldi iteration step 4: normres = 0.2520127946991819
[ Info: Arnoldi iteration step 5: normres = 0.4725000003114789
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.78e-02, 5.28e-02, 3.94e-01, 2.53e-01, 2.02e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3257270154858286
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (8.22e-03, 2.53e-01, 1.51e-01, 1.08e-01, 5.85e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07854270611236563
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.19e-04, 1.75e-02, 7.55e-03, 3.93e-02, 5.89e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11118080897767185
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.54e-05